In [1]:
# conda env : freecad_fem_env

import os
import sys

sys.path.append(os.path.join(os.environ["CONDA_PREFIX"], "lib"))

import FreeCAD
import Part
import ObjectsFem
import Mesh
import MeshPart
import Fem
import femmesh.femmesh2mesh
from femtools import ccxtools
import trimesh
import numpy as np
import datetime
from FreecadFemLib import *
import xlsxwriter


In [2]:
#init
doc = App.newDocument("doc")

trimesh_scene_meshes = []
constraint_scene_meshes = []
view_meshes = True

doc,trimesh_scene_meshes,initMesh = document_setup(doc,trimesh_scene_meshes)


 ******* Mesh.obj is loaded ******* 


--------
invalid_points: False, has_manifolds: False,orient_faces: False, self_intersect: False

--------
mesh result mesh is removed...
shape result mesh is removed...


In [3]:
# for objs in doc.Objects:
#     print(objs.Name)

In [4]:

for state_indx in range(1,2):

    #------  setup the state-action
    print(f"\n\n-----------\n state - {state_indx}\n-----------\n\n")
    force_position = np.random.randint(3,97)
    print(f"force position : {force_position}\n")

    doc , fem_ok = create_shape_femmesh(doc)    
    if not fem_ok: break

    doc , fixed_indx = set_constraint_fixed(doc)
    doc , force_indx = set_constraint_force_placement(doc, force_position=force_position)    
                                            
    if len(fixed_indx)==0 or len(force_indx)==0:
        print("force or fixed constraints are empty... no more analysis will be executed... ")
        break
        
    fixed_scene = add_constraints_to_scene_meshes(doc,fixed_indx,color='blue')
    force_scene = add_constraints_to_scene_meshes(doc,force_indx,color='red')
    constraint_scene_meshes.append(concat_meshes(fixed_scene,force_scene))

    doc = remove_old_shape_result_mesh(doc)
    
    row_data = []
#     for force_val in range(15000,40000,10000):
    for force_val in range(2000,20000,10000):    
      
        doc = remove_old_mesh_result_mesh(doc)
        doc = set_constraint_force_value(doc,force_val)
        # run Analysis
        doc, fem_volume = run_analysis(doc)
        if not fem_volume: break

        #------   create and export result mesh
        doc, out_mesh = create_mesh_from_result(doc)
        if not out_mesh: break
        if not checkMesh(out_mesh): break    

        result_trimesh = Meshobj_to_trimesh(out_mesh)
        result_trimesh_decimated = result_trimesh.simplify_quadratic_decimation(2000)
        trimesh_decimated_topology = trimesh_to_mesh_topology(result_trimesh_decimated)
        result_trimesh_decimated = set_trimesh_color(result_trimesh_decimated)

        if view_meshes : trimesh_scene_meshes.append(result_trimesh_decimated)

        now = str(datetime.datetime.now())
        sav = "_"+now.replace("-","").replace(" ","").replace(":","").replace(".","")
        
        state_info = f"{initMesh}_{force_position}_{force_val}"
        print(f"state_info : {state_info}")
        saved_filename = f"{sav}.obj"
        save_trimesh(result_trimesh_decimated,saved_filename)
        
        row_data.append([f"{initMesh}.obj",force_position,force_val,saved_filename])

    add_to_excel_file(row_data)
#     #------   prepare shape for next state
#     doc = remove_old_femmesh(doc)
#     doc = clear_constraints(doc)
#     doc = remove_old_solid(doc)

#     doc = create_shape_from_mesh(doc,trimesh_decimated_topology)
#     doc, solid = convert_to_solid(doc) 
    
print("done!")



-----------
 state - 1
-----------


force position : 81

num of faces as fixed constraints: 22
num of faces as force constraints: 4
Analysis done successfully...
CCX_Results
Mesh by surface search method:  4.736405
mesh_out is converted to mesh...

--------
invalid_points: False, has_manifolds: False,orient_faces: False, self_intersect: False

--------
state_info : _20210517163105503117_81_2000
file is saved: /scratch/aifa/DATA/FCAD_3D/_20210517165005979203.obj
mesh result mesh is removed...
Analysis done successfully...
CCX_Results
Mesh by surface search method:  3.5777140000000003
mesh_out is converted to mesh...

--------
invalid_points: False, has_manifolds: False,orient_faces: False, self_intersect: False

--------
state_info : _20210517163105503117_81_12000
file is saved: /scratch/aifa/DATA/FCAD_3D/_20210517165010434935.obj
old femmesh is removed...
FemConstraints cleared...
old solid result mesh is removed...
shape from mesh done...
done!


In [5]:
# print('initial mesh:')
# mesh_scene = generate_complete_meshes_scene(doc,trimesh_scene_meshes)
# mesh_scene.show()


In [6]:
# print("the ith state - action (before running the analysis)")
# print(len(trimesh_scene_meshes), len(constraint_scene_meshes))

# i = len(trimesh_scene_meshes)
# state_scene = []

# state_scene.append(trimesh_scene_meshes[0])
# state_scene.append(constraint_scene_meshes[0])

# _scene = generate_complete_meshes_scene(doc,state_scene)
# _scene.show()

In [7]:
# s = scene_view_normals(result_trimesh_decimated)
# s.show()

In [8]:
### save_image() has problem with opengl (and jupyter)

# from PIL import Image
# import io 
# image = np.array(Image.open(io.BytesIO(trimesh.Scene(pnt).save_image(resolution=(200, 200)))))

# trimesh.Scene(pnt).show(line_settings={'point_size':130})